<a href="https://colab.research.google.com/github/abuelnasr0/Semantic-Search-project/blob/populate_vectorDB/Notebooks/vector_database/populate_vdb_with_captions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install grpcio==1.49.1 -q
!pip install pymilvus==2.2.11 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 12.0 MB/s eta 0:00:00


In [6]:
!pip install sentence_transformers -q

In [1]:
uri = ""
token = ""

In [2]:
from pymilvus import connections, Collection

connections.connect("default",
                  uri=uri,
                  token=token)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
PATH = '/content/drive/MyDrive/Graduation_project/Data/Youtube_captions/'

In [5]:
import os
import json
import glob

playlists = []

for filename in glob.glob(os.path.join(PATH+'Captions_with_stride', '*.json')):
    with open(filename, encoding='utf-8', mode='r') as json_file:
      playlist_captions = json.load(json_file)
      playlists.append(playlist_captions)

In [19]:
with open(f"/content/drive/MyDrive/Graduation_project/Data/Youtube_captions/Stored_captions.txt", "r") as outfile:
    stored_captions = outfile.readlines()

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model_name = 'sentence-transformers/msmarco-bert-base-dot-v5'
model = SentenceTransformer(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [13]:
def get_videos_next_id():
  """
  returns the next id to insert in the vector database
  """
  videos_collection = Collection("youtube_videos")
  return videos_collection.num_entities


In [23]:
def store_captions(videos):
  id = get_videos_next_id()
  ids = []
  videos_ids = []
  start_times = []
  texts = []
  for video in videos:
    video_id = video['video_id']
    for caption in video['video_captions'] :
      ids.append(id)
      id+=1
      # embeddings.append(model.encode(caption["text"]).tolist())
      videos_ids.append(video_id)
      start_times.append(str(caption["start"]))
      texts.append(caption["text"])

  embeddings = model.encode(texts).tolist()
  data = [ids, videos_ids, embeddings, start_times, texts]
  videos_collection = Collection("youtube_videos")
  videos_collection.load()
  videos_collection.insert(data)
  videos_collection.flush()
  print(videos_collection.num_entities)

In [24]:
for playlist in playlists:
  if playlist["playlist_id"] in stored_captions :
    continue
  store_captions(playlist["videos"])
  stored_captions.append(playlist["playlist_id"])


with open(f"/content/drive/MyDrive/Graduation_project/Data/Youtube_captions/Stored_captions.txt", "w") as outfile:
    stord_captons = outfile.writelines(stored_captions)

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

778


In [25]:
# Search test
query = "machine learning"
query_embedding = model.encode(query)
query_E = query_embedding.tolist()
top_k = 15

search_params = {"metric_type": "IP"}

videos_collection = Collection("youtube_videos")



results = videos_collection.search(
    data = [query_E],
    anns_field = "embedding",
    param = search_params,
    output_fields=['text', 'start_time', 'video_id'],
    limit = top_k
)

# print(results[0].distances)


print(results[0][0].entity.get("text"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

So natural language processing, computational biology, computer vision robotics all rely today, heavily on machine learning.And you'll see those in those subjects as well.So we're not going to compress five subjects into three lectures.But what we are going to do is give you the introduction.We're going to start by talking about the basic concepts of machine learning.The idea of having examples, and how do you talk about features representing those examples, how do you measure distances between them, and use the notion of distance to try and group similar things together as a way of doing machine learning.And we're going to look, as a consequence, of two different standard ways of doing learning.One, we call classification methods.Example we're going to see, there is something called "k nearest neighbor" and the second class, called clustering methods.Classification works well when I have what we would call labeled data.I know labels on my examples, and I'm going to use that to try and

In [26]:
connections.disconnect("default")